In [1]:
import os
import re
import random
import numpy as np

In [ ]:
import torch
import torch.nn as nn

from PIL import Image
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader

In [ ]:
class MaskDataset(Dataset):
    """

    """

    class MaskLabels:
        MASK = 0
        INCORRECT = 1
        NORMAL = 2

    class GenderLabels:
        MALE = 0
        FEMALE = 1

    class AgeLabels:
        get_label = lambda x: 0 if int(x) < 30 else 1 if int(x) < 60 else 2

    img_extensions = [
        ".jpg", ".JPG", ".jpeg", ".JPEG", ".png", ".PNG"
    ]

    valid_file_name = {
        "mask1": MaskLabels.MASK,
        "mask2": MaskLabels.MASK,
        "mask3": MaskLabels.MASK,
        "mask4": MaskLabels.MASK,
        "mask5": MaskLabels.MASK,
        "incorrect_mask": MaskLabels.INCORRECT,
        "normal": MaskLabels.NORMAL
    }


    # 클래스의 정적 메소드를 정의합니다.

    @staticmethod
    def is_image(file):
        """
        파일이 이미지인지 아닌지를 구분합니다.
        :param file: 이미지 파일
        :return:
        """
        # 파일의 확장자만을 분리하여 저장합니다.
        file_name, file_extension = os.path.splitext(file)

        # 파일의 확장자가 이미지인지 아닌지 여부를 반환합니다.
        return (file_extension in MaskDataset.img_extensions)

    @staticmethod
    def encode_label(age_label, gender_label, mask_label):
        return mask_label * 6 + gender_label * 3 + age_label

    @staticmethod
    def decode_label(class_label):
        mask_label = (class_label // 6) % 3
        gender_label = (class_label // 3) % 2
        age_label = (class_label % 3)
        return mask_label, gender_label, age_label


    # 데이터 셋 모델 인스턴스를 초기에 생성할 때 실행해야 하는 메소드를 정의합니다.

    def __init__(self, data_path, transform = None):
        self.data_path = data_path

        # 데이터의 feature value를 저장합니다.
        self.image_paths = []
        self.age_labels = []
        self.gender_labels = []
        self.mask_labels = []

        if transform is None:
            self.transform = transforms.Compose([transforms.ToTensor()])
        else:
            self.set_transform(transform)

        self.set_up()

    def set_up(self):
        data_dir = self.data_path
        dir_list = os.listdir(data_dir)

        for directory in dir_list:
            # "."로 시작하는 파일 및 폴더는 무시합니다.
            if re.match('^[.]', directory):
                continue

            image_dir = os.path.join(data_dir, directory)
            for image in os.listdir(image_dir):
                # 이미지가 맞는지 아닌지 확인합니다.
                if not self.is_image(image):
                    continue

                image_name, _ = os.path.splitext(image)
                # 이미지 파일명이 유효한지 확인합니다.
                if image_name not in MaskDataset.valid_file_name:
                    continue

                # 데이터 별로 존재하는 이미지 폴더명을 분리하여 데이터를 얻습니다.
                id, gender, race, age = directory.split('_')
                image_path = os.path.join(image_dir, image)
                age_label = MaskDataset.AgeLabels.get_label(age)
                gender_label = getattr(MaskDataset.AgeLabels, gender.upper())
                mask_label = MaskDataset.valid_file_name[image_name]

                self.image_paths.append(image_path)
                self.age_labels.append(age_label)
                self.gender_labels.append(gender_label)
                self.mask_labels.append(mask_label)



    def set_transform(self, transform):
        self.transform = transform


    # 필요한 데이터를 가져오는 데 필요한 메소드를 정의합니다.

    def __getitem__(self, index):
        image = self.get_image(index)
        age_label = self.get_age_label(index)
        gender_label = self.gender_labels(index)
        mask_label = self.mask_labels(index)

    def get_image(self, index):
        return Image.open(self.image_paths[index])

    def get_age_label(self, index):
        return self.age_labels[index]

    def get_gender_label(self, index):
        return self.gender_labels[index]

    def get_mask_label(self, index):
        return self.mask_labels[index]




